In [ ]:
import os, sys
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import ROOT
import root_numpy as rn
from larocv import larocv

rs     = ['run','subrun']
rse    = ['run','subrun','event']
rsev   = ['run','subrun','event','vtxid']
rserv  = ['run','subrun','event','roid','vtxid']

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
name='inclusive_elec_vertex_09152017_p00'
LLCUT=-15.125

all_df = pd.read_pickle(os.path.join("..","ll_bin","{}_all.pkl".format(name)))
LL_df = pd.read_pickle(os.path.join("..","ll_bin","{}_post_LL.pkl".format(name)))
FILE=""
POTFILE=""

if 'inclusive_elec' in name:
    FILE = "/data/vgenty/tufts/inclusive_elec/ll_inter_ana.root"
    POTFILE = "/home/vgenty/sw/larcv/app/LArOpenCVHandle/ana/pot/pot_inclusive_elec.root"

if 'inclusive_muon' in name:
    FILE = "/data/vgenty/tufts/inclusive_muon/comb_ana_ll.root"
    POTFILE = "/home/vgenty/sw/larcv/app/LArOpenCVHandle/ana/pot/pot_inclusive_muon.root"

if 'ncpizero' in name:
    FILE = "/data/vgenty/tufts/ncpizero/comb_ana_ll.root"
    POTFILE = "/home/vgenty/sw/larcv/app/LArOpenCVHandle/ana/pot/pot_ncpizero.root"
    
mcinfo_df = pd.DataFrame(rn.root2array(FILE,treename='MCINFO'))
mcinfo_df = mcinfo_df.set_index(rse)

pot_df = pd.DataFrame(rn.root2array(POTFILE,treename='pot_tree'))
pot_df = pot_df.set_index(rs)

In [ ]:
nu_group = all_df.groupby(rse).nth(0)
LL_group = LL_df.set_index(rse)
nu_group['ecount'] = nu_group.reset_index().groupby(["run","subrun"])["event"].transform("count").values

res_comb    = pd.concat([nu_group,mcinfo_df],axis=1,join_axes=[nu_group.index])
res_comb_LL = pd.concat([LL_group,mcinfo_df],axis=1,join_axes=[LL_group.index])

res_comb_pot = pd.concat([res_comb.reset_index().set_index(rs),pot_df],axis=1,join_axes=[res_comb.reset_index().set_index(rs).index])

res_comb_pot['pot'] = res_comb_pot['pot'] / res_comb_pot['ecount']

res_comb_pot = res_comb_pot.reset_index()

In [ ]:
print res_comb_pot.index.size
print res_comb_pot[res_comb_pot.apply(lambda x : np.isnan(x['pot'])==False,axis=1)].index.size
print res_comb_pot[res_comb_pot.apply(lambda x : np.isnan(x['pot'])==False and np.isnan(x['mcinfoEnergy'])==False,axis=1)].index.size

In [ ]:
print "total POT",pot_df.pot.values.sum(),pot_df.index.size
print "total POT",res_comb_pot['pot'].values.sum(),res_comb_pot.index.size

df0 = res_comb_pot[np.isfinite(res_comb_pot['pot'])]
df1 = df0[np.isfinite(df0['mcinfoEnergy'])]

print "total POT 0==",df0['pot'].values.sum(),df0.index.size
print "total POT 1==",df1['pot'].values.sum(),df1.index.size

res_comb_pot_corr = df1.copy()

In [ ]:
print len(res_comb_pot.groupby(['run','subrun']))
print len(df0.groupby(['run','subrun']))
print len(df1.groupby(['run','subrun']))

In [ ]:
print res_comb_pot_corr.index.size
print res_comb_pot_corr.query("mcinfoMinDToEdge>17").index.size 
print float(res_comb_pot_corr.query("mcinfoMinDToEdge>17").index.size) / float(res_comb_pot_corr.index.size)

In [ ]:
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'

import collections
code_to_name = collections.OrderedDict()
code_to_name[1000] = "MEC"
code_to_name[1001] = "CCQE"
code_to_name[1002] = "NCQE"
code_to_name[1003] = "ResCCNuProtonPiPlus"
code_to_name[1004] = "ResCCNuNeutronPi0"
code_to_name[1005] = "ResCCNuNeutronPiPlus"
code_to_name[1006] = "ResNCNuProtonPi0"
code_to_name[1007] = "ResNCNuProtonPiPlus"
code_to_name[1008] = "ResNCNuNeutronPi0"
code_to_name[1009] = "ResNCNuNeutronPiMinus"
code_to_name[1013] = "ResNCNuBarProtonPi0"
code_to_name[1014] = "ResNCNuBarProtonPiPlus"
code_to_name[1015] = "ResNCNuBarNeutronPi0"
code_to_name[1091] = "CCDIS"
code_to_name[1092] = "NCDIS"
code_to_name[1096] = "NCCOH"
code_to_name[1097] = "CCCOH"

def get_cmap(n, name='hsv'):
    return plt.cm.get_cmap(name, n)

cmap = get_cmap(len(code_to_name.keys()))

ix=0
code_2_name = collections.OrderedDict()
for key in code_to_name:
    code_2_name[key] = (code_to_name[key],cmap(ix))
    ix += 1


iy=-1
for df in [res_comb_pot_corr,
           #res_comb_pot_corr.query("num_croi>0"),
           #res_comb_pot_corr.query("good_croi_ctr>0"),
           res_comb_LL.query("scedr<5 & LL>@LLCUT")]:
    
    print iy,LLCUT
    iy+=1
    
        
    fig,ax = plt.subplots(figsize=(10,6))
    data_v = []
    inter_v = []
    ratio_v = []
    weights_v = []
    scale = 6.6*10**20
    cnt = 0
    bins=np.arange(0,5000+100,100)
    color_v=[]
    for name, group in df.groupby("mcinfoInteractionType"):
        data = group.mcinfoEnergy.values * 1000.0
        cnt += data.size
        weights = [scale / res_comb_pot_corr.pot.values.sum()] * data.size
        label = "%s N=%.02f" % (code_2_name[name][0],data.size * weights[0])
        inter_v.append(label)
        data_v.append(data)
        weights_v.append(weights)
        color_v.append(code_2_name[name][1])

    c = ax.hist(data_v,
                weights=weights_v,
                bins=bins,
                stacked=True,
                histtype='stepfilled',
                alpha=1.0,
                label=inter_v,
                color=color_v,
                lw=2)
    
    ax.hist([],histtype='step',color='white',alpha=0.0,label='Total: %.01f' % (c[0][-1].sum()))
    print "raw events",cnt
    print "total events",c[0][-1].sum()
    print [c[0][0].sum()] + [cc.sum() - c[0][ix-1].sum() for ix,cc in enumerate(c[0]) if ix>0]

    legend=ax.legend(fontsize=15)
    ax.set_xlabel("True Neutrino Energy [MeV]",fontweight='bold')
    ax.set_ylabel("Events",fontweight='bold')
    ax.set_title("All inclusive_elec [6.6e20 POT]",fontweight='bold')

    ax.grid()
    #legend.remove()
    #ax.set_xlim(200,800)
    ax.set_ylim(0,130)
    print "iy",iy
    if iy==1:
        ax.set_ylim(0,20)
        ax.set_title("",fontweight='bold')
        
    plt.savefig("inclusive_all_E_breakdown_inclusive_elec_%d.pdf" % iy)
    plt.show()


In [ ]:
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'

fix,ax=plt.subplots(figsize=(10,6))

LLCUT=-15.125
Xmin = 200
Xmax = 800
dX   = 50

bins=np.arange(Xmin,Xmax + dX,dX)
SS = 'selected1L1P==1 & mcinfoInteractionType==1001 & mcinfoMinDToEdge>10'
data=res_comb_pot.groupby(rse).nth(0).query(SS)['energyInit'].values
weights = [scale / res_comb_pot_corr.pot.values.sum()] * data.size
#weights = [1.0]*data.size
ax.hist(data,bins=bins,weights=weights,color='red',histtype='stepfilled',alpha=0.1)
sig = ax.hist(data,bins=bins,weights=weights,color='red',label='CCQE 1L1P 10cm FV',histtype='step',lw=3)

SS += "& good_croi_ctr>0"
data=res_comb_pot.groupby(rse).nth(0).query(SS)['energyInit'].values
weights = [scale / res_comb_pot_corr.pot.values.sum()] * data.size
#weights = [1.0]*data.size
ax.hist(data,bins=bins,weights=weights,color='magenta',histtype='stepfilled',alpha=0.1)
ax.hist(data,bins=bins,weights=weights,color='magenta',label='Good cROI',histtype='step',lw=3)

SS+= '& LL>=@LLCUT'
data=res_comb_LL.query(SS)['energyInit'].values
weights = [scale / res_comb_pot_corr.pot.values.sum()] * data.size
#weights = [1.0]*data.size
ax.hist(data,bins=bins,weights=weights,color='green',histtype='stepfilled',alpha=0.1)
ax.hist(data,bins=bins,weights=weights,color='green',label='Pass Nue LL',histtype='step',lw=3)

SS+= '& scedr<5'
data=res_comb_LL.query(SS)['energyInit'].values
weights = [scale / res_comb_pot_corr.pot.values.sum()] * data.size
#weights = [1.0]*data.size
ax.hist(data,bins=bins,weights=weights,color='blue',alpha=0.1,histtype='stepfilled')
reco = ax.hist(data,bins=bins,weights=weights,color='blue',label='Pass Nue LL & Good Vertex',histtype='step',lw=3)

ax.set_ylim(0,22)
#ax.set_ylim(0,220)
ax.set_xlim(200,800)
ax.grid()
ax.legend(loc='upper left',fontsize=18)

ax.set_ylabel("Events",fontweight='bold')
ax.set_xlabel("True Neutrino Energy [MeV]",fontweight='bold')
plt.savefig("00_E.pdf")
plt.show()


print sig[0].sum()
print reco[0].sum()


In [ ]:
fig,ax = plt.subplots(figsize=(10,6))
    
reco_sig = reco[0] / sig[0]
reco_sig = np.nan_to_num(reco_sig)
bidx     = np.nonzero(reco_sig)
signal_v = sig[0]
param_v  = sig[1][:-1] + float(dX)/2.0
eff_v    = reco_sig

res_v     = np.where(eff_v==0)[0]
eff_vv    = []
signal_vv = []
param_vv  = []
eff_v_    = []
signal_v_ = []
param_v_  = []

for ix in xrange(eff_v.size):
    if ix in res_v:
        eff_vv.append(np.array(eff_v_))
        signal_vv.append(np.array(signal_v_))
        param_vv.append(np.array(param_v_))
        eff_v_    = []
        signal_v_ = []
        param_v_  = []
        continue

    eff_v_.append(eff_v[ix])
    signal_v_.append(signal_v[ix])
    param_v_.append(param_v[ix])

eff_vv.append(np.array(eff_v_))
signal_vv.append(np.array(signal_v_))
param_vv.append(np.array(param_v_))

for param_v_,eff_v_,signal_v_ in zip(param_vv,eff_vv,signal_vv):
    ax.plot(param_v_,eff_v_,'o',color='blue',markersize=8)
    ax.errorbar(param_v_,eff_v_,yerr= np.sqrt( eff_v_ * ( 1 - eff_v_ ) / signal_v_ ),lw=2,color='blue')

ax.set_ylabel("Overall Efficiency",fontweight='bold')
ax.set_xlabel("True Neutrino Energy [MeV]",fontweight='bold')
ax.set_ylim(0,1.0)
plt.grid()
plt.savefig("00_eff.pdf")
plt.show()

In [ ]:
eff_vv[0].mean()
